
Чтобы создать аналитический дашборд на основе Streamlit с данными из базы данных PostgreSQL, необходимо выполнить следующие шаги:

Настроить подключение к базе данных PostgreSQL.
Создать и запустить Streamlit приложение.
Визуализировать данные и создать интерактивные элементы.
Шаг 1: Настройка подключения к базе данных PostgreSQL
Перед началом убедитесь, что у вас установлены необходимые библиотеки: psycopg2, pandas, и streamlit.

Установите библиотеки (если они еще не установлены):

In [ ]:
pip install psycopg2-binary pandas streamlit


Создание Streamlit приложения
Создайте файл, например dashboard.py, и начните с импорта необходимых библиотек и настройки подключения к базе данных.

In [ ]:

import streamlit as st
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Настройка подключения к базе данных PostgreSQL
DB_HOST = 'your_db_host'
DB_NAME = 'your_db_name'
DB_USER = 'your_db_user'
DB_PASS = 'your_db_password'

# Создание подключения к базе данных
@st.cache(allow_output_mutation=True)
def get_db_connection():
    engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    return engine

# Загрузка данных из базы данных
def load_data():
    engine = get_db_connection()
    query = "SELECT * FROM your_table_name"  # Измените на ваше название таблицы
    df = pd.read_sql(query, engine)
    return df

# Основная функция для создания дашборда
def main():
    st.title("Аналитический дашборд")

    # Загрузка данных
    df = load_data()

    # Вычисление метрик
    station_load = df['станция входа'].value_counts() + df['станция выхода'].value_counts()
    average_capacity = station_load.mean()
    station_capacity_percent = (station_load / average_capacity) * 100
    entry_count = df['станция входа'].value_counts()
    exit_count = df['станция выхода'].value_counts()
    df['время входа'] = pd.to_datetime(df['время входа'])
    df['время выхода'] = pd.to_datetime(df['время выхода'])
    df['длительность поездки'] = df['время выхода'] - df['время входа']
    average_trip_duration = df['длительность поездки'].mean()

    # Отображение метрик
    st.header("Основные метрики")
    st.metric("Средняя пропускная способность", f"{average_capacity:.2f}")
    st.metric("Средняя продолжительность поездки", f"{average_trip_duration}")

    # Визуализация данных
    st.header("Загруженность станции")
    st.bar_chart(station_load)

    st.header("Пропускная способность станции (%)")
    st.bar_chart(station_capacity_percent)

    st.header("Количество входов на станции")
    st.bar_chart(entry_count)

    st.header("Количество выходов на станции")
    st.bar_chart(exit_count)

    # Отображение данных в таблице
    st.header("Данные по поездкам")
    st.dataframe(df)

if __name__ == "__main__":
    main()


Объяснение кода:
Подключение к базе данных: Используем SQLAlchemy для подключения к PostgreSQL.
Загрузка данных: Загрузка данных из базы данных и их кэширование с помощью @st.cache для улучшения производительности.
Вычисление метрик: Вычисляем основные метрики, такие как загруженность станций, пропускная способность, количество входов и выходов, и среднюю продолжительность поездки.
Визуализация данных: Используем Streamlit для создания заголовков, метрик и графиков для визуализации данных.
Запуск приложения: Запускаем Streamlit приложение для отображения дашборда.
С помощью этого подхода можно создать интерактивный аналитический дашборд, который будет автоматически загружать данные из базы данных PostgreSQL, обновляться в реальном времени и предоставлять удобные инструменты для анализа данных.